In [33]:
import betfairlightweight
from   betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json

Login etc...

In [34]:
my_username = "darrenlogue"
my_password = "Fangman123!"
my_app_key  = "LwpwussYFAn9kO7f"

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key)

trading.login_interactive()

<LoginResource>

Get the event types i.e. Sports, ids etc

In [35]:
results = trading.betting.list_event_types()

In [36]:
for i in results:
    print(i.event_type.id, i.event_type.name, i.market_count)

1 Soccer 12055
2 Tennis 601
3 Golf 7
4 Cricket 372
5 Rugby Union 80
1477 Rugby League 1
6 Boxing 25
7 Horse Racing 793
8 Motor Sport 2
27454571 Esports 52
10 Special Bets 25
998917 Volleyball 18
11 Cycling 1
2152880 Gaelic Games 1
6422 Snooker 36
6423 American Football 269
7511 Baseball 1
451485 Winter Sports 18
7522 Basketball 295
7524 Ice Hockey 189
61420 Australian Rules 5
468328 Handball 67
3503 Darts 258
26420387 Mixed Martial Arts 56
4339 Greyhound Racing 88
2378961 Politics 49


list events using a market filter...

In [37]:
racing_filter = filters.market_filter(text_query="Horse Racing")
results = trading.betting.list_events(filter=racing_filter)

for i in results:
    print(i.event.id, i.event.name, i.market_count)

31133019 Burnie (AUS) 17th Dec 14
31103839 Ascot 18th Dec 2
31131612 Lismore (AUS) 17th Dec 16
31132107 Uttoxeter 17th Dec 35
31131599 Eagle Farm (AUS) 18th Dec 18
31133774 Ascot 18th Dec 6
31120332 Leopardstown 28th Dec 1
30362199 2022 Antepost 4
30222167 Kempton 26th Dec 1
12713050 Daily Win Dist Odds 1
31119041 Randwick (AUS) 2nd Apr 2
31131589 Gloucester Park (AUS) 17th Dec 20
31034744 Flemington (AUS) 1st Nov 1
31132536 Charles Town (US) 16th Dec 8
31132540 Remington Park (US) 16th Dec 9
31131507 Moonee Valley (AUS) 17th Dec 16
31132530 Delta Downs (US) 16th Dec 7
31133814 Wolverhampton 18th Dec 7
31132533 Turf Paradise (US) 16th Dec 16
31131508 Randwick (AUS) 18th Dec 20
31132523 Aqueduct (US) 16th Dec 8
31132522 Gulfstream Park (US) 16th Dec 12
31133802 Newcastle 18th Dec 7
31132136 Southwell 17th Dec 39
31132513 Laurel Park (US) 16th Dec 10
31131547 Yarra Valley (AUS) 17th Dec 16
31119001 Caulfield (AUS) 26th Feb 1
31131551 Albany (AUS) 17th Dec 18
31133855 Deauville (FRA) 17th

Get static market data using listMarketCatalogue...

In [38]:
racing_filter = filters.market_filter(event_type_ids=[7],  
                                      market_countries=["GB"],
                                      market_type_codes=["WIN"])

market_projection=["RUNNER_DESCRIPTION", 
                   "RUNNER_METADATA", 
                   "COMPETITION", 
                   "EVENT", 
                   "EVENT_TYPE", 
                   "MARKET_DESCRIPTION", 
                   "MARKET_START_TIME"]

results = trading.betting.list_market_catalogue(market_projection=market_projection,
                                                filter=racing_filter,
                                                max_results=100)
for i in results:
    print("{0} {1:d}:{2:02d} {3} ({4})".format(i.market_id, i.market_start_time.hour, 
                                               i.market_start_time.minute, i.event.venue,  
                                               i.description.market_type))

1.192433827 11:40 Uttoxeter (WIN)
1.192433988 11:50 Southwell (WIN)
1.192433832 12:15 Uttoxeter (WIN)
1.192433993 12:25 Southwell (WIN)
1.192433769 12:35 Ascot (WIN)
1.192433837 12:50 Uttoxeter (WIN)
1.192433998 13:00 Southwell (WIN)
1.192433774 13:10 Ascot (WIN)
1.192433842 13:25 Uttoxeter (WIN)
1.192434003 13:35 Southwell (WIN)
1.192433779 13:45 Ascot (WIN)
1.192433847 14:00 Uttoxeter (WIN)
1.192434008 14:10 Southwell (WIN)
1.192433784 14:20 Ascot (WIN)
1.192433852 14:35 Uttoxeter (WIN)
1.192434013 14:45 Southwell (WIN)
1.192433789 14:55 Ascot (WIN)
1.192433857 15:10 Uttoxeter (WIN)
1.192434018 15:20 Southwell (WIN)
1.192433794 15:30 Ascot (WIN)
1.192434023 15:50 Southwell (WIN)
1.192434746 16:15 Kempton (WIN)
1.192434751 16:45 Kempton (WIN)
1.192434756 17:15 Kempton (WIN)
1.192434761 17:45 Kempton (WIN)
1.192434766 18:15 Kempton (WIN)
1.192434771 18:45 Kempton (WIN)
1.192434776 19:15 Kempton (WIN)
1.192434781 19:45 Kempton (WIN)
1.192468672 11:50 Haydock (WIN)
1.192468439 12:00 Newc

Dynamic market price request using listMarketBook...

Change the id (and rem there might not be any more races today)

In [39]:
price_projection = filters.price_projection(price_data=filters.price_data(ex_all_offers=True))

market_books = trading.betting.list_market_book(market_ids=["1.192433827"], price_projection=price_projection)

for market_book in market_books:
    print(market_book.market_id, market_book.inplay, market_book.status, market_book.total_matched)
    for runner in market_book.runners:  
        print(runner.selection_id, runner.status, runner.last_price_traded, runner.total_matched)

1.192433827 False OPEN 209.6
41707918 ACTIVE 2.74 0.0
28182160 ACTIVE 4.7 0.0
10216476 ACTIVE 5.0 0.0
41731455 ACTIVE None 0.0
42091524 ACTIVE None 0.0
42091523 ACTIVE 28.0 0.0
38612565 ACTIVE 12.5 0.0
36585609 ACTIVE None 0.0
39127721 ACTIVE None 0.0
41731485 ACTIVE None 0.0
37471778 REMOVED None None


#### Place Order

In [51]:
market_id    = "1.192433827"
selection_id = 41707918
limit_order  = filters.limit_order(size=2.00, price=1.01, persistence_type="LAPSE")

instruction  = filters.place_instruction(order_type="LIMIT", 
                                         selection_id=selection_id, 
                                         side="LAY", 
                                         limit_order=limit_order)

place_orders = trading.betting.place_orders(market_id=market_id, instructions=[instruction])
print(place_orders.status)

for order in place_orders.place_instruction_reports:
    print("Status: {0}, BetId: {1}, Average Price Matched: {2}".format(order.status,  
                                                                       order.bet_id,  
                                                                       order.average_price_matched))

SUCCESS
Status: SUCCESS, BetId: 253763382184, Average Price Matched: 0.0


#### Update Order

In [52]:
bet_id = 253763382184
instruction  = filters.update_instruction(bet_id=bet_id, new_persistence_type="PERSIST")
update_order = trading.betting.update_orders(market_id=market_id, instructions=[instruction])

print(update_order.status)
for order in update_order.update_instruction_reports:
    print("Status: {0}".format(order.status))

SUCCESS
Status: SUCCESS


#### Replace Order

In [53]:
instruction   = filters.replace_instruction(bet_id=bet_id, new_price=1.10)
replace_order = trading.betting.replace_orders(market_id=market_id, instructions=[instruction])

for order in replace_order.replace_instruction_reports:
    place_report  = order.place_instruction_reports
    cancel_report = order.cancel_instruction_reports
    print("Status: {0}, New BetId: {1}, Average Price Matched: {2}".format(order.status, 
                                                                           place_report.bet_id, 
                                                                           place_report.average_price_matched))

Status: SUCCESS, New BetId: 253763422603, Average Price Matched: 0.0


#### Cancel Order

In [54]:
bet_id = 253763422603

instruction  = filters.cancel_instruction(bet_id=bet_id, size_reduction=2.00)
cancel_order = trading.betting.cancel_orders(market_id=market_id, instructions=[instruction])

for cancel in cancel_order.cancel_instruction_reports:
    print("Status: {0}, Size Cancelled: {1}, Cancelled Date: {2}".format(cancel.status,  
                                                                         cancel.size_cancelled, 
                                                                         cancel.cancelled_date))

Status: SUCCESS, Size Cancelled: 2.0, Cancelled Date: 2021-12-16 19:56:21
